In [ ]:
#import numpy
#from isicarchive import jitfunc
from isicarchive.api import IsicApi
#from isicarchive.font import Font
#from isicarchive.sampler import Sampler, _sample_grid_2d

username = 'weberj3@mskcc.org'

doc_folder = 'C:\\Users\\weberj3\\Documents\\'
cache_folder = doc_folder + 'ISIC'

debug = True

api = IsicApi(username, cache_folder=cache_folder, debug=debug)

In [ ]:
from matplotlib import pyplot
%matplotlib inline

image = api.image('ISIC_0000000')
image.load_image_data()
rimage = api.resample_image(image.data, (1200,1600))
api.set_text_in_image(rimage, 'This is merely a test...', fsize=0.04, min_alpha=1.0, color=[0,0,0], bcolor=[255,255,255])

pyplot.figure(figsize=(12,8))
pyplot.imshow(rimage)
pyplot.show()

In [ ]:
k = [k for k in api.image_cache.keys()]

In [ ]:
api.image_cache[k[12419]]

In [ ]:
from typing import Any
from isicarchive import func
print(func.parse_expr('$_id.-1$/image_$_id$_$name$.jpg', api.image_cache[k[134]]))

In [ ]:
api.select_images([])
len(api.image_selection)

In [ ]:
api.image_selection[0]

In [ ]:
from isicarchive import func
abbr = func.getxattr(api._features, '[].abbreviation')

In [ ]:
t = api.set_text_in_image(None, '\n'.join(abbr), fsize=48.0,bcolor=[255,255,255], color=[0,0,0])
t

In [ ]:
from matplotlib import pyplot
%matplotlib inline

pyplot.figure(figsize=(16,24))
pyplot.imshow(t[4][0:2000,:,:])
pyplot.show()

In [ ]:
api._features